python虚拟机是stack base，以栈为基础

## CodeObject

In [1]:
def double(x):
    return x * 2

In [2]:
'''
定义函数：即定义一个function object(函数对象)保存在变量中(从code object中load，load本身函数和名称)
'''
double

<function __main__.double>

In [3]:
import dis
dis.dis(double)

  2           0 LOAD_FAST                0 (x)
              2 LOAD_CONST               1 (2)
              4 BINARY_MULTIPLY
              6 RETURN_VALUE


In [4]:
dis.dis("max(1,1)")

  1           0 LOAD_NAME                0 (max)
              2 LOAD_CONST               0 (1)
              4 LOAD_CONST               0 (1)
              6 CALL_FUNCTION            2
              8 RETURN_VALUE


In [5]:
'''
python中所有对象都可以调用，如果对象是callable则正常load，然后进行调用；
                        如果对象并非callable，如下面的1，可以正常load对象，然后会在call_function的runtime时报错
'''
dis.dis("1()")


  1           0 LOAD_CONST               0 (1)
              2 CALL_FUNCTION            0
              4 RETURN_VALUE


In [6]:
'''
所以函数可以作为参数，传入其他函数
'''
def cal_number(func, x):
    print(func(x))
cal_number(double, 2)


4


In [7]:
'''
函数不光可以作为变量传入，也可以作为返回值传出
'''
def get_multiple_func(n):
    def multiple(x):
        return n * x
    return multiple
double = get_multiple_func(2) #2为n，传给multiple，然后返回multiple(x):return x *2
triple = get_multiple_func(3)

print(double(5))
print(triple(5))

10
15


In [8]:
def f():
    pass
print(f.__code__)

<code object f at 0x10ced75d0, file "<ipython-input-8-847fb160c0f0>", line 1>


## decorator装饰器

In [25]:
def decorator(f):
    pass

In [26]:
dis.dis(decorator)

  2           0 LOAD_CONST               0 (None)
              2 RETURN_VALUE


In [27]:
@decoartor
def double(x):
    return x * 2
dis.dis(double)

  1           0 LOAD_NAME                0 (dis)
              2 LOAD_ATTR                0 (dis)
    -->       4 LOAD_NAME                1 (decorator)
              6 CALL_FUNCTION            1
              8 PRINT_EXPR
             10 LOAD_CONST               0 (None)
             12 RETURN_VALUE


In [28]:
'''
以上等价于
'''
double4 = decorator(double)

In [29]:
dis.dis(double4)

  1           0 LOAD_NAME                0 (dis)
              2 LOAD_ATTR                0 (dis)
    -->       4 LOAD_NAME                1 (decorator)
              6 CALL_FUNCTION            1
              8 PRINT_EXPR
             10 LOAD_CONST               0 (None)
             12 RETURN_VALUE


In [36]:
'''
实际上decorator的入参、返回都是函数
'''

import time

def timeit(f): #入参f函数
    def wrapper(x): #入参x
        start = time.time()
        ret = f(x)  #调用f(入参为x)的函数变量
        print("运行时间:", time.time() - start)
        return ret #返回函数结果
    return wrapper #返回wrapper函数

#等价于 my_funct = timeit(my_funct)  ==> wrapper中的f为myfunct
@timeit
def my_func(x):
    time.sleep(x)

my_func(1)

运行时间: 1.0042610168457031


In [37]:
@timeit
def other_func(x):
    return x * 2
print(other_func(3))

运行时间: 9.5367431640625e-07
6


In [68]:
def timeit2(f):
    '''
    *args：把所有的参数按出现顺序打包成list
    **kargs：把所有的key=value形式的参数打包成一个dict
    '''
    def wrapper(*args, **kwargs): # *
        start = time.time()
        ret = f(*args, **kwargs)  #调用f(入参为x)的函数变量
        print("运行时间:", time.time() - start)
        return ret #返回函数结果
    
    return wrapper #返回wrapper函数

@timeit2
def my_func(x):
    time.sleep(x)
    
@timeit2
def my_add(x,y):
    return x+y
print(my_func(2))
print("===========================")
print(my_add(2,3))
print("===========================")

运行时间: 2.004676103591919
None
运行时间: 2.1457672119140625e-06
5


In [73]:
#带参数的decorator
def timeit3(iteration):
    
    def inner(f):
        
        def wrapper(*args, **kwargs): # *
            start = time.time()
            ret = 0
            for _ in range(iteration):
                ret += f(*args, **kwargs)  #调用f(入参为x)的函数变量
            print("运行时间:", time.time() - start)
            return ret #返回函数结果
    
        return wrapper #返回wrapper函数
    
    return inner

'''
不带参数的@timeit会直接调用double
带参数10的@timeit会先调用timeit(10)，然后返回一个函数inner，用这个函数再调用double
'''

@timeit3(1000)
def double(x):
    return x * 2

double(1)
#等价于 double = timeit3(10)(double)
#=====================================
#等价于 inner = timeit3(10)
# 加上 double = inner(double)



运行时间: 0.00022792816162109375


2000

In [67]:
#*args,  **kwargs用法
def a(*args,  **kwargs):
    print("======*args========")
    print(args)
    print("=====**kwargs======")
    print(kwargs)

In [63]:
a(*[3,4,5])
# 或者直接a(3,4,5)，默认进*args

======*args========
(3, 4, 5)
=====**kwargs======
{}


In [60]:
a(**{"a":3,"b":4,"c":5})

======*args========
()
=====**kwargs======
{'a': 3, 'b': 4, 'c': 5}


辅助解释：https://www.jb51.net/article/134782.htm

In [ ]:
def 